In [43]:
from pathlib import Path
import re
import pickle
from datetime import datetime

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from scipy.ndimage import distance_transform_edt, gaussian_filter1d, binary_fill_holes
from skimage.measure import label, regionprops
from skimage.graph import route_through_array
from skimage.morphology import binary_dilation, binary_erosion, skeletonize

from topostats.plottingfuncs import Colormap
from topostats.tracing.splining import windowTrace
from topostats.measure.feret import get_feret_from_mask
from topostats.measure.curvature import discrete_angle_difference_per_nm_linear

colormap = Colormap()
CMAP = colormap.get_cmap()
COLOUR_PATH_CMAP = mpl.cm.coolwarm

VMIN = 0
VMAX = 4

In [ ]:
TODAY_DATE = datetime.now().strftime("%Y-%m-%d")
DATE_TO_READ_FROM = "2024-03-22"
BASE_DATA_DIR = Path(f"/Users/sylvi/topo_data/hariborings/extracted_grains/")
data_dirs = {
    "ON_SC": BASE_DATA_DIR / f"cas9_ON_SC/{DATE_TO_READ_FROM}",
    "OT1_SC": BASE_DATA_DIR / f"cas9_OT1_SC/{DATE_TO_READ_FROM}",
    "OT2_SC": BASE_DATA_DIR / f"cas9_OT2_SC/{DATE_TO_READ_FROM}",
}
BASE_SAVE_DIR = Path(f"/Users/sylvi/topo_data/hariborings/processed_grains/")
MAX_PX_TO_NM = 10.0

save_dirs = {
    "ON_SC": BASE_SAVE_DIR / f"cas9_ON_SC/{TODAY_DATE}",
    "OT1_SC": BASE_SAVE_DIR / f"cas9_OT1_SC/{TODAY_DATE}",
    "OT2_SC": BASE_SAVE_DIR / f"cas9_OT2_SC/{TODAY_DATE}",
}
for sample_type in save_dirs.keys():
    save_dirs[sample_type].mkdir(exist_ok=True, parents=True)

sample_types = ["ON_SC", "OT1_SC", "OT2_SC"]
grains_dicts = {}
for sample_type in sample_types:
    grains_dicts[sample_type] = {}
    file_path = data_dirs[sample_type] / "grain_dict.pkl"
    with open(file_path, "rb") as f:
        sample_grains_dicts = pickle.load(f)
        for grain_index, single_grain_dict in sample_grains_dicts.items():
            p_to_nm = single_grain_dict["p_to_nm"]
            if p_to_nm > MAX_PX_TO_NM:
                print(
                    f"Skipping grain {grain_index} for sample type [{sample_type}] due to p_to_nm too large: {p_to_nm} > {MAX_PX_TO_NM}"
                )
            else:
                grains_dicts[sample_type][grain_index] = single_grain_dict


# print num grains for each sample type
for sample_type, grain_dict in grains_dicts.items():
    print(f"Number of grains for sample type [{sample_type}]: {len(grain_dict.keys())}")

In [46]:
def plot_all_grains_dictionary(
    grain_dict: dict,
    num_cols: int = 3,
    vmin: float = VMIN,
    vmax: float = VMAX,
    cmap: mpl.cm = CMAP,
    plot_paths: list[str] = None,
    plot_colour_paths: list[tuple[str, str]] = None,
    save_path: Path | None = None,
    show_plot: bool = True,
    dpi=200,
) -> None:
    """plot grains in a grain dictioanry in a grid layout"""
    num_grains = len(grain_dict.keys())
    num_rows = num_grains // num_cols + (num_grains % num_cols > 0)
    num_images_per_grain = 2  # image and mask
    fig, ax = plt.subplots(
        nrows=num_rows,
        ncols=num_cols * num_images_per_grain,
        figsize=(num_cols * 4, num_rows * 4),
        constrained_layout=True,
        dpi=200,
    )
    for i, (grain_index, grain_data) in enumerate(grain_dict.items()):
        grain_image = grain_data["image"]
        grain_mask = grain_data["predicted_mask"]
        grain_p_to_nm = grain_data["p_to_nm"]

        row = i // num_cols
        col = (i % num_cols) * num_images_per_grain

        image_ax = ax[row, col]
        image_ax.imshow(grain_image, vmin=vmin, vmax=vmax, cmap=cmap)
        image_ax.set_title(f"Grain {grain_index}\np_to_nm: {grain_p_to_nm:.2f} nm/px")
        image_ax.axis("off")

        # plot mask
        mask_ax = ax[row, col + 1]
        mask_ax.imshow(grain_mask)
        # plot paths if present and required
        if plot_paths is not None:
            for path_name in plot_paths:
                if path_name in grain_data:
                    path = grain_data[path_name]
                    if path is not None:
                        mask_ax.plot(path[:, 1], path[:, 0], color="red", linewidth=2, label=path_name)
        if plot_colour_paths is not None:
            for path_name, colour_array in plot_colour_paths:
                if path_name in grain_data:
                    path = grain_data[path_name]
                    cvals = grain_data[colour_array]
                    normalised_cvals = (cvals - np.min(cvals)) / (np.max(cvals) - np.min(cvals))
                    if path is not None:
                        for point_index, point in enumerate(path):
                            colour = COLOUR_PATH_CMAP(normalised_cvals[point_index])
                            if point_index > 0:
                                previous_point = path[point_index - 1]
                                mask_ax.plot(
                                    [previous_point[1], point[1]],
                                    [previous_point[0], point[0]],
                                    color=colour,
                                    linewidth=2,
                                )

        mask_ax.set_title("Mask")
        mask_ax.axis("off")

    plt.legend(loc="upper right", bbox_to_anchor=(1.1, 1), fontsize="small")

    if save_path is not None:
        # create the parent directory
        save_path.parent.mkdir(parents=True, exist_ok=True)
        print(f"saving figure to {save_path}")
        fig.savefig(save_path, dpi=dpi)

    if show_plot:
        plt.show()
    else:
        plt.close(fig)

In [ ]:
# plot all grains
for sample_type, grain_dict in grains_dicts.items():
    print(f"Plotting grains for sample type [{sample_type}]")
    plot_all_grains_dictionary(grain_dict, vmin=VMIN, vmax=VMAX, cmap=CMAP)
    plt.savefig(save_dirs[sample_type] / f"grains_{sample_type}.png", dpi=200)
    plt.close()

In [ ]:
# pathfinding
grains_dicts_paths = {}
for sample_type, grains_dict_sample in grains_dicts.items():
    grains_dicts_paths[sample_type] = {}
    for grain_index, grain_data in grains_dict_sample.items():
        grain_image = grain_data["image"]
        grain_mask = grain_data["predicted_mask"]
        intersection_labels = grain_data["intersection_labels"]

        # distance transforms
        distance_transform = distance_transform_edt(grain_mask > 0)
        distance_transform[grain_mask == 2] = 0

        # start at point where intersection region 0 has maximum distance transform
        intersection_labels = label(intersection_labels)
        intersection_regions = regionprops(intersection_labels)
        # Get intersection regions
        region_0 = intersection_regions[0]
        region_1 = intersection_regions[1]
        # Create list of distance transform values for each region
        region_0_distance_transform_values = []
        region_1_distance_transform_values = []
        for pixel in region_0.coords:
            region_0_distance_transform_values.append(distance_transform[pixel[0], pixel[1]])
        for pixel in region_1.coords:
            region_1_distance_transform_values.append(distance_transform[pixel[0], pixel[1]])
        region_0_distance_transform_values = np.array(region_0_distance_transform_values)
        region_1_distance_transform_values = np.array(region_1_distance_transform_values)
        # Get the maximum distance transform value for each region
        region_0_max_distance_transform_value = np.max(region_0_distance_transform_values)
        region_1_max_distance_transform_value = np.max(region_1_distance_transform_values)
        region_0_max_distance_transform_value_index = np.argmax(region_0_distance_transform_values)
        region_1_max_distance_transform_value_index = np.argmax(region_1_distance_transform_values)
        # get the pixel coord of the maximum distance transform value for each region
        region_0_max_distance_transform_pixel = region_0.coords[region_0_max_distance_transform_value_index]
        region_1_max_distance_transform_pixel = region_1.coords[region_1_max_distance_transform_value_index]

        start_point = (region_0_max_distance_transform_pixel[0], region_0_max_distance_transform_pixel[1])
        end_point = (region_1_max_distance_transform_pixel[0], region_1_max_distance_transform_pixel[1])

        # invert the distance transform to get the cost map
        cost_map = np.max(distance_transform) - distance_transform
        # set the maximum value to be huge
        cost_map[cost_map == np.max(cost_map)] = 1e4

        route, weight = route_through_array(array=cost_map, start=start_point, end=end_point)

        route = np.array(route)

        # save the original grain data
        grains_dicts_paths[sample_type][grain_index] = grain_data
        # add the route to the grain data
        grains_dicts_paths[sample_type][grain_index]["path"] = route

# plot all grains with paths
for sample_type, grain_dict in grains_dicts_paths.items():
    print(f"Plotting grains with paths for sample type [{sample_type}]")
    plot_all_grains_dictionary(
        grain_dict=grain_dict,
        vmin=VMIN,
        vmax=VMAX,
        cmap=CMAP,
        plot_paths=["path"],
        save_path=save_dirs[sample_type] / f"grains_with_paths_{sample_type}.png",
    )

In [ ]:
# smoothing the paths
grains_dicts_smooth_paths = {}
for sample_type, grains_dict_sample in grains_dicts_paths.items():
    grains_dicts_smooth_paths[sample_type] = {}
    for grain_index, grain_data in grains_dict_sample.items():
        grain_image = grain_data["image"]
        grain_mask = grain_data["predicted_mask"]
        intersection_labels = grain_data["intersection_labels"]
        p_to_nm = grain_data["p_to_nm"]
        path = grain_data["path"]

        smoothed_path = windowTrace.pool_trace_linear(
            pixel_trace=path,
            rolling_window_size=5,
            pixel_to_nm_scaling=p_to_nm,
        )

        # save the original grain data
        grains_dicts_smooth_paths[sample_type][grain_index] = grain_data
        # add the smoothed path to the grain data
        grains_dicts_smooth_paths[sample_type][grain_index]["smoothed_path"] = smoothed_path

# plot all grains with smoothed paths
for sample_type, grain_dict in grains_dicts_smooth_paths.items():
    print(f"Plotting grains with smoothed paths for sample type [{sample_type}]")
    plot_all_grains_dictionary(
        grain_dict=grain_dict,
        vmin=VMIN,
        vmax=VMAX,
        cmap=CMAP,
        plot_paths=["smoothed_path"],
        save_path=save_dirs[sample_type] / f"grains_with_smoothed_paths_{sample_type}.png",
        show_plot=False,
    )

In [40]:
# calculate the feret diameter for each grain
grains_dicts_feret_diameter = {}
for sample_type, grains_dict_sample in grains_dicts_smooth_paths.items():
    grains_dicts_feret_diameter[sample_type] = {}
    for grain_index, grain_data in grains_dict_sample.items():
        grain_image = grain_data["image"]
        grain_mask = grain_data["predicted_mask"]
        intersection_labels = grain_data["intersection_labels"]
        p_to_nm = grain_data["p_to_nm"]
        smoothed_path = grain_data["smoothed_path"]

        grain_mask_dna_only = grain_mask.copy()
        grain_mask_dna_only[grain_mask == 2] = 0

        # calculate the feret diameter
        feret_diameters = get_feret_from_mask(mask_im=grain_mask_dna_only)
        min_feret = feret_diameters["min_feret"] * p_to_nm
        max_feret = feret_diameters["max_feret"] * p_to_nm

        # save the original grain data
        grains_dicts_feret_diameter[sample_type][grain_index] = grain_data
        # add the feret diameter to the grain data
        grains_dicts_feret_diameter[sample_type][grain_index]["min_feret"] = min_feret
        grains_dicts_feret_diameter[sample_type][grain_index]["max_feret"] = max_feret

In [ ]:
# calcuate curvature of the path
grains_dicts_curvature = {}
for sample_type, grains_dict_sample in grains_dicts_feret_diameter.items():
    grains_dicts_curvature[sample_type] = {}
    for grain_index, grain_data in grains_dict_sample.items():
        grain_image = grain_data["image"]
        grain_mask = grain_data["predicted_mask"]
        intersection_labels = grain_data["intersection_labels"]
        p_to_nm = grain_data["p_to_nm"]
        smoothed_path = grain_data["smoothed_path"]

        # calculate the curvature
        path_curvatures = discrete_angle_difference_per_nm_linear(trace_nm=smoothed_path * p_to_nm)

        # save the original grain data
        grains_dicts_curvature[sample_type][grain_index] = grain_data
        # add the curvature to the grain data
        grains_dicts_curvature[sample_type][grain_index]["curvature"] = path_curvatures


# plot all grains with curvature
for sample_type, grain_dict in grains_dicts_curvature.items():
    print(f"Plotting grains with curvature for sample type [{sample_type}]")
    plot_all_grains_dictionary(
        grain_dict=grain_dict,
        vmin=VMIN,
        vmax=VMAX,
        cmap=CMAP,
        plot_colour_paths=[("smoothed_path", "curvature")],
        save_path=save_dirs[sample_type] / f"grains_with_curvature_{sample_type}.png",
        show_plot=True,
        dpi=400,
    )

Plotting grains with curvature for sample type [ON_SC]


/var/folders/sr/wjtfqr9s6x3bw1s647t649x80000gn/T/ipykernel_51694/858372027.py:68: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  plt.legend(loc="upper right", bbox_to_anchor=(1.1, 1), fontsize="small")


saving figure to /Users/sylvi/topo_data/hariborings/processed_grains/cas9_ON_SC/2025-06-23/grains_with_curvature_ON_SC.png


ValueError: Image size of 9600x137600 pixels is too large. It must be less than 2^16 in each direction.